In [1]:
# Cell 1: Environment and Imports
import os
import sys
from dotenv import load_dotenv

# Add project to path
sys.path.append('.')

# Load environment
load_dotenv()

# Import our LSTM components
from config.settings import LSTMConfig, AstraDBConfig
from core.gate_networks import LSTMGateNetwork
from core.memory_manager import MemoryManager
from models.memory_item import MemoryItem
from models.rl_experience import RLExperience, RLState, RLAction, RLReward
from database.astra_connector import AstraDBConnector
from rl_training.gate_optimizer import GateOptimizer
from rl_training.reward_calculator import RewardCalculator
from utils.similarity import cosine_similarity
from utils.token_counter import estimate_tokens

# Validate environment
DATABASE_ID = os.getenv('ASTRA_DATABASE_ID')
REGION = os.getenv('ASTRA_REGION') 
TOKEN = os.getenv('ASTRA_TOKEN')
KEYSPACE = os.getenv('ASTRA_KEYSPACE', 'memory_db')

if not all([DATABASE_ID, REGION, TOKEN]):
    raise ValueError("Missing Astra DB credentials in .env file!")

print("✅ All LSTM components imported successfully")
print(f"🔗 Database: {DATABASE_ID}")
print(f"🌍 Region: {REGION}")
print(f"🔑 Keyspace: {KEYSPACE}")

✅ All LSTM components imported successfully
🔗 Database: cb844ee7-37f7-453e-9ece-e0720040b3df
🌍 Region: us-east1
🔑 Keyspace: memory_db


In [2]:
# Cell 2: Initialize LSTM System
async def initialize_lstm_system():
    """Initialize the complete LSTM memory system"""
    
    # Create configuration
    config = LSTMConfig()
    config.astra_db.endpoint = f"https://{DATABASE_ID}-{REGION}.apps.astra.datastax.com"
    config.astra_db.token = TOKEN
    config.astra_db.keyspace = KEYSPACE
    
    # Initialize memory manager
    memory_manager = MemoryManager(config)
    await memory_manager.initialize()
    
    # Initialize RL trainer
    rl_trainer = GateOptimizer(
        gate_network=memory_manager.gate_network,
        learning_rate=0.001
    )
    
    return memory_manager, rl_trainer, config

# Run initialization
memory_manager, rl_trainer, config = await initialize_lstm_system()
print("✅ LSTM system initialized successfully")
print(f"📊 Gate thresholds: {memory_manager.gate_network.get_thresholds()}")

INFO:core.memory_manager:Initializing Memory Manager...
INFO:database.astra_connector:Connecting to Astra DB: https://cb844ee7-37f7-453e-9ece-e0720040b3df-us-east1.apps.astra.datastax.com
INFO:astrapy.data.database:findCollections
INFO:httpx:HTTP Request: POST https://cb844ee7-37f7-453e-9ece-e0720040b3df-us-east1.apps.astra.datastax.com/api/json/v1/memory_db "HTTP/1.1 200 OK"
INFO:astrapy.data.database:finished findCollections
INFO:database.astra_connector:Connected! Found collections: []
INFO:database.astra_connector:Successfully connected to Astra DB
INFO:core.memory_manager:No existing gate network found, using fresh model
INFO:core.memory_manager:Memory Manager initialized successfully


✅ LSTM system initialized successfully
📊 Gate thresholds: {'forget': 0.30000001192092896, 'input': 0.4000000059604645, 'output': 0.4000000059604645}


In [3]:
# Cell 3: Test Memory Creation
import random
# Sample journal entries
sample_entries = [
    "I had an amazing meeting with Sarah today about the new AI project. Super excited about the possibilities!",
    "Feeling stressed about the upcoming deadline. Need to organize my time better.",
    "Learned something fascinating about neural networks today. The attention mechanism is incredible.",
    "Had lunch with my friend. We talked about his new job and he seems happy.",
    "Dentist appointment tomorrow at 2pm. Really nervous about it."
]

# Process entries through LSTM system
print("📝 Processing journal entries through LSTM gates...")
created_memories = []

for i, entry in enumerate(sample_entries, 1):
    print(f"\n📖 Entry {i}: '{entry[:50]}...'")
    
    # Generate dummy feature vector (replace with Component 4)
    feature_vector = [random.random() for _ in range(90)]
    
    # Generate dummy embeddings (replace with Component 3)  
    embeddings = [random.random() for _ in range(768)]
    
    # Process through memory manager
    memory = await memory_manager.process_new_entry(
        user_id="test_user_123",
        feature_vector=feature_vector,
        content=entry,
        embeddings=embeddings
    )
    
    if memory:
        created_memories.append(memory)
        print(f"✅ Memory created: {memory.memory_type} (importance: {memory.importance_score:.3f})")
    else:
        print("❌ Entry not stored (low importance)")

print(f"\n📊 Results: {len(created_memories)}/{len(sample_entries)} entries became memories")

INFO:core.memory_manager:Processing new entry for user test_user_123
INFO:core.memory_manager:Input gate score: 0.456, decision: True
INFO:astrapy.data.collection:insertOne on 'memory_embeddings'
ERROR:database.astra_connector:Error saving memory: Cannot encode a datetime without timezone information ('tzinfo'). See the APIOptions.SerdesOptions.accept_naive_datetimes setting if you want to relax this write-time safeguard.
ERROR:database.memory_store:Failed to save memory 860deb82-b709-4742-abcd-7c56f6a6a119
ERROR:core.memory_manager:Failed to save memory for user test_user_123
INFO:core.memory_manager:Processing new entry for user test_user_123
INFO:core.memory_manager:Input gate score: 0.553, decision: True
INFO:astrapy.data.collection:insertOne on 'memory_embeddings'
ERROR:database.astra_connector:Error saving memory: Cannot encode a datetime without timezone information ('tzinfo'). See the APIOptions.SerdesOptions.accept_naive_datetimes setting if you want to relax this write-time s

📝 Processing journal entries through LSTM gates...

📖 Entry 1: 'I had an amazing meeting with Sarah today about th...'
❌ Entry not stored (low importance)

📖 Entry 2: 'Feeling stressed about the upcoming deadline. Need...'
❌ Entry not stored (low importance)

📖 Entry 3: 'Learned something fascinating about neural network...'
❌ Entry not stored (low importance)

📖 Entry 4: 'Had lunch with my friend. We talked about his new ...'
❌ Entry not stored (low importance)

📖 Entry 5: 'Dentist appointment tomorrow at 2pm. Really nervou...'
❌ Entry not stored (low importance)

📊 Results: 0/5 entries became memories


In [ ]:
# Cell 4: Test Context Retrieval
print("🔍 Testing memory context retrieval...")

# Test queries
test_queries = [
    "How's work going?",
    "Tell me about AI",
    "Any upcoming appointments?",
    "How am I feeling lately?"
]

for query in test_queries:
    print(f"\n❓ Query: '{query}'")
    
    # Generate query features (placeholder)
    query_features = [random.random() for _ in range(90)]
    
    # Get relevant context
    relevant_memories, metadata = await memory_manager.get_relevant_context(
        user_id="test_user_123",
        query=query,
        query_features=query_features,
        max_tokens=1500
    )
    
    print(f"🎯 Found {len(relevant_memories)} relevant memories")
    print(f"📊 Metadata: {metadata}")
    
    if relevant_memories:
        for memory in relevant_memories[:2]:  # Show top 2
            print(f"   💭 {memory.memory_type}: {memory.content_summary[:60]}...")
            print(f"      Importance: {memory.importance_score:.3f}")

In [ ]:
# Cell 5: RL Training Setup
print("🎓 Setting up RL training...")

# Create reward calculator
reward_calc = RewardCalculator()

# Simulate user feedback for training
def simulate_user_interaction(memories_used: List[MemoryItem]) -> Dict:
    """Simulate user interaction data"""
    return {
        'response_length': random.randint(50, 200),
        'has_follow_up_questions': random.choice([True, False]),
        'session_duration_minutes': random.uniform(2, 15),
        'positive_feedback': random.choice([True, False]),
        'context_relevance_score': random.uniform(0.3, 1.0),
        'memories_used': len(memories_used),
        'processing_time_seconds': random.uniform(0.5, 2.0)
    }

# Test reward calculation
print("🏆 Testing reward calculation...")

for i in range(3):
    # Simulate interaction
    interaction_data = simulate_user_interaction(created_memories[:3])
    
    # Calculate reward using our reward calculator
    reward = reward_calc.calculate_simple_reward({
        'user_engagement': random.uniform(0.4, 1.0),
        'context_relevance': random.uniform(0.3, 1.0), 
        'memory_efficiency': random.uniform(0.5, 1.0),
        'user_satisfaction': random.uniform(0.4, 1.0),
        'conversation_quality': random.uniform(0.5, 1.0)
    })
    
    print(f"🎯 Interaction {i+1}: Reward = {reward:.3f}")
    print(f"   Engagement metrics: {interaction_data}")

print("✅ Reward calculation working")